In [1]:
import pandas as pd

bitfinex_pairs = ['bateth', 'batusd', 'bchbtc', 'bchusd', 'bchusdt', 'btcusd',
                  'btcusdt', 'dashbtc', 'dashusd', 'eosbtc', 'eosusd',
                  'eosusdt', 'etcbtc', 'etcusd', 'ethbtc', 'ethusd', 'gntusd',
                  'ltcbtc', 'ltcusd', 'ltcusdt', 'manausd', 'repbtc',
                  'repusd', 'xlmbtc', 'xlmusd', 'xrpbtc', 'xrpusd', 'zecbtc',
                  'zecusd', 'zrxbtc', 'zrxusd']

coinbase_pro_pairs = ['bateth', 'batusdc', 'bchbtc', 'bchusd', 'btcusd', 
                      'btcusdc', 'dashbtc', 'dashusd', 'eosbtc', 'eosusd',
                      'etcbtc', 'etcusd', 'ethbtc', 'ethusd', 'ethusdc',
                      'gntusdc', 'ltcbtc', 'ltcusd', 'manausdc', 'repbtc',
                      'repusd', 'xlmbtc', 'xlmusd', 'xrpbtc', 'xrpusd',
                      'zecbtc', 'zecusdc', 'zrxbtc', 'zrxusd']

poloniex_pairs = ['bateth', 'batusdt', 'bchbtc', 'bchusdc', 'bchusdt', 
                  'btcusdc', 'btcusdt', 'dashbtc', 'dashusdc', 'dashusdt',
                  'eosbtc', 'eosusdc', 'eosusdt', 'etcbtc', 'etcusdc', 
                  'etcusdt', 'ethbtc', 'ethusdc', 'ethusdt', 'gntusdt', 
                  'ltcbtc', 'ltcusdc', 'ltcusdt', 'manausdt', 'repbtc',
                  'repusdt', 'xlmbtc', 'xlmusdc', 'xlmusdt', 'xrpbtc',
                  'xrpusdc', 'xrpusdt', 'zecbtc', 'zecusdc', 'zecusdt', 
                  'zrxbtc', 'zrxusdt']

tables = {}
columns = ['time', 'open', 'high', 'low', 'close', 'volume']
tables['bitfinex'] = {pair: pd.DataFrame(columns=columns
                                        ) for pair in bitfinex_pairs}
tables['coinbase-pro'] = {pair: pd.DataFrame(columns=columns
                                            ) for pair in coinbase_pro_pairs}
tables['poloniex'] = {pair: pd.DataFrame(columns=columns
                                        ) for pair in poloniex_pairs}

In [2]:
import time
import requests

base_url = ('https://api.cryptowat.ch/markets/{exchange}/{trading_pair}/' +
            'ohlc?apikey={api_key}&periods={periods}&after={after}')
api_key = 'INSERT_PUBLIC_KEY_HERE'
periods = '60'
after = round(time.time()) - 200

start = time.time()
while time.time() < start + 200:
    for exchange in tables:
        for trading_pair in tables[exchange]:
            url = base_url.format(exchange=exchange,
                                  trading_pair=trading_pair,
                                  api_key=api_key,
                                  periods=periods,
                                  after=after)
            
            try:
                response = requests.get(url).json()
                candles = response['result'][periods]
                
                timestamps = tables[exchange][trading_pair]['time'].tolist()
                closed_candles = candles[:-1]
                for candle in closed_candles:
                    if candle[0] not in timestamps:
                        new_row = pd.DataFrame([candle[:6]], columns=columns)
                        tables[exchange][trading_pair] = tables[
                            exchange][trading_pair].append(
                            new_row, ignore_index=True)
                
                timestamps = tables[exchange][trading_pair]['time'].tolist()
                latest_candle = closed_candles[-1]
                if latest_candle[0] + int(periods) < time.time():
                    if candle[0] not in timestamps:
                        new_row = pd.DataFrame([latest_candle[:6]],
                                               columns=columns)
                        tables[exchange][trading_pair] = tables[
                            exchange][trading_pair].append(
                            new_row, ignore_index=True)    
            except:
                pass
                
    time.sleep(60)

In [3]:
tables['coinbase-pro']['btcusd']

,time,open,high,low,close,volume
0,1571249760,7969.67,7974.74,7969.67,7974.74,0.771051
1,1571249820,7974.73,7974.73,7970.91,7971.14,2.224526
2,1571249880,7971.14,7971.14,7967.88,7968.63,2.751760
3,1571249940,7968.63,7968.85,7968.19,7968.70,3.523304
4,1571250000,7968.70,7970.85,7968.69,7970.75,1.963131
5,1571250060,7970.75,7970.75,7969.16,7969.16,4.520283


In [4]:
tables['bitfinex']['btcusd']

,time,open,high,low,close,volume
0,1571249760,7999.600000,7999.600000,7999.574797,7999.600000,1.583886
1,1571249820,7999.600000,7999.600000,7999.574797,7999.600000,0.165856
2,1571249880,7999.594092,7999.594092,7999.594092,7999.594092,0.029611
3,1571249940,7999.500000,7999.574797,7999.300000,7999.318098,1.066390
4,1571250000,7999.400000,7999.400000,7999.300000,7999.318098,0.256967
5,1571250060,7999.300000,7999.318098,7999.300000,7999.318098,0.147715


In [5]:
tables['poloniex']['btcusdt']

,time,open,high,low,close,volume
0,1571249760,7974.273027,7974.373027,7974.173027,7974.173027,0.428167
1,1571249820,7974.373027,7974.373027,7974.373027,7974.373027,0.013204
2,1571249880,7974.073026,7974.073026,7971.200000,7971.200000,0.233860
3,1571249940,7971.200000,7971.200000,7971.200000,7971.200000,0.000570
4,1571250000,7971.200000,7971.200000,7971.200000,7971.200000,0.001474
